In [ ]:
import os
import glob
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from mapper_functions import plot_global_tight_pcm, plot_tb_tight_pcm, plot_NA_tight_pcm


import sys;       sys.path.append('../util/shared/python/')
from read_GEOSldas          import read_tilecoord


In [ ]:
expt_name = '1e_LS_DAv8_M36_0'

start_date = datetime(2002, 10, 1)
end_date = datetime(2003, 10, 1)

start_date_str = start_date.strftime('%Y/%m/%d')
end_date_str = end_date.strftime('%Y/%m/%d')

# Define the path directory
root_directory = f'/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/snow_qc_expts/1e_LS_DAv8_M36_0/{expt_name}/output/SMAP_EASEv2_M36_GLOBAL/cat/ens0000'

In [ ]:
# List all tavg24 files in the root directory
files = glob.glob(os.path.join(root_directory,f'{expt_name}.tavg24_1d_lnd_Nt*.nc'))

# Filter files based on the date range
filtered_files = []
for file in files:
    # Extract the date part from the filename
    filename = os.path.basename(file)
    print(f'Processing file: {filename}')
    date_str = filename.split('.')[-2]  # Extract the date part (e.g., 200401)
    file_date = datetime.strptime(date_str, '%Y%m')

    # Check if the file date is within the range
    if start_date <= file_date < end_date:
        filtered_files.append(file)

# Load the data 
ds = xr.open_mfdataset(filtered_files, combine='nested', concat_dim="time")

# Sort the dataset by the time dimension
ds = ds.sortby('time')

# list all the increments files in the root directory (look like: 1e_LS_DAv8_M36_0.monthly_snow_increments.200610.nc)
increment_files = glob.glob(os.path.join(root_directory, f'{expt_name}.monthly_snow_increments*.nc'))
# Filter files based on the date range
filtered_increment_files = []
for file in increment_files:
    # Extract the date part from the filename
    filename = os.path.basename(file)
    print(f'Processing file: {filename}')
    date_str = filename.split('.')[-2]  # Extract the date part (e.g., 200610)
    file_date = datetime.strptime(date_str, '%Y%m')

    # Check if the file date is within the range
    if start_date <= file_date < end_date:
        filtered_increment_files.append(file)
# Load the increments data
ds_increments = xr.open_mfdataset(filtered_increment_files, combine='nested', concat_dim="time")
# Sort the increments dataset by the time dimension
ds_increments = ds_increments.sortby('time')

In [ ]:
# Variables in the dataset
variables = {
'sm_surface': 'SFMC',
'sm_rootzone': 'RZMC',
'sm_profile': 'PRMC',
'precipitation_total_surface_flux': 'PRECTOTCORRLAND',
'vegetation_greenness_fraction': 'GRN',
'leaf_area_index': 'LAI',
'snow_mass': 'SNOMASLAND',
'surface_temperature_of_land_incl_snow': 'TSURFLAND',
'soil_temperature_layer_1': 'TSOIL1',
'snowfall_land': 'PRECSNOCORRLAND',
'snow_depth_within_snow_covered_area_fraction_on_land': 'SNODPLAND',
'snowpack_evaporation_latent_heat_flux_on_land': 'LHLANDSBLN',
'overland_runoff_including_throughflow': 'RUNSURFLAND',
'baseflow_flux_land': 'BASEFLOWLAND',
'snowmelt_flux_land': 'SMLAND',
'total_evaporation_land': 'EVLAND',
'net_shortwave_flux_land': 'SWLAND',
'total_water_storage_land': 'TWLAND',
'fractional_area_of_snow_on_land': 'FRLANDSNO'  # New variable added
}

ftc = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/land_sweeper/' \
      'LS_OLv8_M36/output/SMAP_EASEv2_M36_GLOBAL/rc_out/LS_OLv8_M36.ldas_tilecoord.bin'
tc = read_tilecoord(ftc)
n_tile = tc['N_tile']
lat = tc['com_lat']
lon = tc['com_lon']
area = tc['area']

map_array = np.empty([n_tile, 3])
map_array.fill(np.nan)
map_array[:, 1] = lon
map_array[:, 2] = lat

In [ ]:
# Extract the time array
time_array = ds['time'].values  # This is a numpy datetime64 array

# Convert numpy datetime64 to Python datetime
time_array = time_array.astype('datetime64[M]').astype(object)

# Calculate the number of seconds in each month
seconds_in_month = []
for t in time_array:
    # Start of the current month
    start_of_month = datetime(t.year, t.month, 1)
    # Start of the next month
    if t.month == 12:
        start_of_next_month = datetime(t.year + 1, 1, 1)
    else:
        start_of_next_month = datetime(t.year, t.month + 1, 1)
    # Calculate the number of seconds in the month
    seconds = (start_of_next_month - start_of_month).total_seconds()
    seconds_in_month.append(seconds)

seconds_in_month = np.array(seconds_in_month)
# Reshape seconds_in_month to match the time dimension of var_data
seconds_in_month = seconds_in_month[:, np.newaxis]  # Shape becomes (12, 1)

In [ ]:
expt_name = '1e_LS_OLv8_M36_0'

# Define the path directory
root_directory = f'/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/snow_qc_expts/1e_LS_DAv8_M36_0/{expt_name}/output/SMAP_EASEv2_M36_GLOBAL/cat/ens0000'

# List all tavg24 files in the root directory
files = glob.glob(os.path.join(root_directory, f'{expt_name}.tavg24_1d_lnd_Nt*.nc'))

# Filter files based on the date range
filtered_files = []
for file in files:
    # Extract the date part from the filename
    filename = os.path.basename(file)
    date_str = filename.split('.')[-2]  # Extract the date part (e.g., 200401)
    file_date = datetime.strptime(date_str, '%Y%m')

    # Check if the file date is within the range
    if start_date <= file_date < end_date:
        filtered_files.append(file)

# Load the data 
ds_ol = xr.open_mfdataset(filtered_files, combine='nested', concat_dim="time")

# Sort the dataset by the time dimension
ds_ol = ds_ol.sortby('time')

In [ ]:
expt_name = '1e_LS_DAv8_M36_qc6'

# Define the path directory
root_directory = f'/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/snow_qc_expts/1e_LS_DAv8_M36_0/{expt_name}/output/SMAP_EASEv2_M36_GLOBAL/cat/ens0000'

# List all tavg24 files in the root directory
files = glob.glob(os.path.join(root_directory, f'{expt_name}.tavg24_1d_lnd_Nt*.nc'))

# Filter files based on the date range
filtered_files = []
for file in files:
    # Extract the date part from the filename
    filename = os.path.basename(file)
    date_str = filename.split('.')[-2]  # Extract the date part (e.g., 200401)
    file_date = datetime.strptime(date_str, '%Y%m')

    # Check if the file date is within the range
    if start_date <= file_date < end_date:
        filtered_files.append(file)

# Load the data 
ds_da_old = xr.open_mfdataset(filtered_files, combine='nested', concat_dim="time")

# Sort the dataset by the time dimension
ds_da_old = ds_da_old.sortby('time')

# list all the increments files in the root directory (look like: 1e_LS_DAv8_M36_0.monthly_snow_increments.200610.nc)
increment_files = glob.glob(os.path.join(root_directory, f'{expt_name}.monthly_snow_increments*.nc'))
# Filter files based on the date range
filtered_increment_files = []
for file in increment_files:
    # Extract the date part from the filename
    filename = os.path.basename(file)
    date_str = filename.split('.')[-2]  # Extract the date part (e.g., 200610)
    file_date = datetime.strptime(date_str, '%Y%m')

    # Check if the file date is within the range
    if start_date <= file_date < end_date:
        filtered_increment_files.append(file)
# Load the increments data
ds_da_old_increments = xr.open_mfdataset(filtered_increment_files, combine='nested', concat_dim="time")
# Sort the increments dataset by the time dimension
ds_da_old_increments = ds_da_old_increments.sortby('time')

In [ ]:
ol_scf = ds_ol[variables['fractional_area_of_snow_on_land']].values
da_scf = ds[variables['fractional_area_of_snow_on_land']].values
do_scf = ds_da_old[variables['fractional_area_of_snow_on_land']].values

ol_evap = ds_ol[variables['total_evaporation_land']].values * seconds_in_month
da_evap = ds[variables['total_evaporation_land']].values * seconds_in_month
do_evap = ds_da_old[variables['total_evaporation_land']].values * seconds_in_month

ol_runoff = ds_ol[variables['overland_runoff_including_throughflow']].values * seconds_in_month
da_runoff = ds[variables['overland_runoff_including_throughflow']].values * seconds_in_month
do_runoff = ds_da_old[variables['overland_runoff_including_throughflow']].values * seconds_in_month

ol_snowmelt = ds_ol[variables['snowmelt_flux_land']].values * seconds_in_month
da_snowmelt = ds[variables['snowmelt_flux_land']].values * seconds_in_month
do_snowmelt = ds_da_old[variables['snowmelt_flux_land']].values * seconds_in_month

ol_baseflow = ds_ol[variables['baseflow_flux_land']].values * seconds_in_month
da_baseflow = ds[variables['baseflow_flux_land']].values * seconds_in_month
do_baseflow = ds_da_old[variables['baseflow_flux_land']].values * seconds_in_month

ol_snowmass = ds_ol[variables['snow_mass']].values
da_snowmass = ds[variables['snow_mass']].values
do_snowmass = ds_da_old[variables['snow_mass']].values

ol_snowdepth = ds_ol[variables['snow_depth_within_snow_covered_area_fraction_on_land']].values
da_snowdepth = ds[variables['snow_depth_within_snow_covered_area_fraction_on_land']].values

ol_snowfall = ds_ol[variables['snowfall_land']].values * seconds_in_month
da_snowfall = ds[variables['snowfall_land']].values * seconds_in_month

ol_soil_moisture = ds_ol[variables['sm_surface']].values
da_soil_moisture = ds[variables['sm_surface']].values
do_soil_moisture = ds_da_old[variables['sm_surface']].values

ol_rzmc = ds_ol[variables['sm_rootzone']].values
da_rzmc = ds[variables['sm_rootzone']].values
do_rzmc = ds_da_old[variables['sm_rootzone']].values

ol_sm_profile = ds_ol[variables['sm_profile']].values
da_sm_profile = ds[variables['sm_profile']].values
do_sm_profile = ds_da_old[variables['sm_profile']].values

ol_evap_sum = np.nansum(ol_evap, axis=0)
da_evap_sum = np.nansum(da_evap, axis=0)
do_evap_sum = np.nansum(do_evap, axis=0)

ol_runoff_sum = np.nansum(ol_runoff, axis=0)
da_runoff_sum = np.nansum(da_runoff, axis=0)
do_runoff_sum = np.nansum(do_runoff, axis=0)

ol_snowmelt_sum = np.nansum(ol_snowmelt, axis=0)
da_snowmelt_sum = np.nansum(da_snowmelt, axis=0)
do_snowmelt_sum = np.nansum(do_snowmelt, axis=0)

ol_baseflow_sum = np.nansum(ol_baseflow, axis=0)
da_baseflow_sum = np.nansum(da_baseflow, axis=0)

ol_soil_moisture_mean = np.nanmean(ol_soil_moisture, axis=0)
da_soil_moisture_mean = np.nanmean(da_soil_moisture, axis=0)
do_soil_moisture_mean = np.nanmean(do_soil_moisture, axis=0)

ol_rzmc_mean = np.nanmean(ol_rzmc, axis=0)
da_rzmc_mean = np.nanmean(da_rzmc, axis=0)
do_rzmc_mean = np.nanmean(do_rzmc, axis=0)

ol_sm_profile_mean = np.nanmean(ol_sm_profile, axis=0)
da_sm_profile_mean = np.nanmean(da_sm_profile, axis=0)
do_sm_profile_mean = np.nanmean(do_sm_profile, axis=0)

ol_precip = ds_ol[variables['precipitation_total_surface_flux']].values * seconds_in_month
da_precip = ds[variables['precipitation_total_surface_flux']].values * seconds_in_month
do_precip = ds_da_old[variables['precipitation_total_surface_flux']].values * seconds_in_month

ol_precip_sum = np.nansum(ol_precip, axis=0)
da_precip_sum = np.nansum(da_precip, axis=0)
do_precip_sum = np.nansum(do_precip, axis=0)

ol_tws = ds_ol[variables['total_water_storage_land']].values
da_tws = ds[variables['total_water_storage_land']].values
do_tws = ds_da_old[variables['total_water_storage_land']].values
ol_tws_mean = np.nanmean(ol_tws, axis=0)
da_tws_mean = np.nanmean(da_tws, axis=0)
do_tws_mean = np.nanmean(do_tws, axis=0)

da_total_incr_sum = ds_increments['TOTAL_INCREMENT_SUM'].values
do_total_incr_sum = ds_da_old_increments['TOTAL_INCREMENT_SUM'].values

da_total_incr_cnt = ds_increments['TOTAL_INCREMENT_EVENT_COUNT'].values
do_total_incr_cnt = ds_da_old_increments['TOTAL_INCREMENT_EVENT_COUNT'].values


In [ ]:
sm_mask = np.logical_or(ol_snowmelt_sum > 0, da_snowmelt_sum > 0, do_snowmelt_sum > 0)
sm_mask = sm_mask.T

map_array[:, 0] = sm_mask

plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n Snowmelt mask', '-')

In [ ]:
ol_snowmelt = np.where(sm_mask, ol_snowmelt, np.nan)
da_snowmelt = np.where(sm_mask, da_snowmelt, np.nan)
do_snowmelt = np.where(sm_mask, do_snowmelt, np.nan)
ol_evap = np.where(sm_mask, ol_evap, np.nan)
da_evap = np.where(sm_mask, da_evap, np.nan)
do_evap = np.where(sm_mask, do_evap, np.nan)
ol_runoff = np.where(sm_mask, ol_runoff, np.nan)
da_runoff = np.where(sm_mask, da_runoff, np.nan)
do_runoff = np.where(sm_mask, do_runoff, np.nan)
ol_baseflow = np.where(sm_mask, ol_baseflow, np.nan)
da_baseflow = np.where(sm_mask, da_baseflow, np.nan)
do_baseflow = np.where(sm_mask, do_baseflow, np.nan)
ol_precip = np.where(sm_mask, ol_precip, np.nan)
da_precip = np.where(sm_mask, da_precip, np.nan)
do_precip = np.where(sm_mask, do_precip, np.nan)
ol_tws = np.where(sm_mask, ol_tws, np.nan)
da_tws = np.where(sm_mask, da_tws, np.nan)
do_tws = np.where(sm_mask, do_tws, np.nan)
da_total_incr_sum = np.where(sm_mask, da_total_incr_sum, np.nan)
do_total_incr_sum = np.where(sm_mask, do_total_incr_sum, np.nan)
da_total_incr_cnt = np.where(sm_mask, da_total_incr_cnt, np.nan)
do_total_incr_cnt = np.where(sm_mask, do_total_incr_cnt, np.nan)

ol_soil_moisture = np.where(sm_mask, ol_soil_moisture, np.nan)
da_soil_moisture = np.where(sm_mask, da_soil_moisture, np.nan)
do_soil_moisture = np.where(sm_mask, do_soil_moisture, np.nan)
ol_sm_profile = np.where(sm_mask, ol_sm_profile, np.nan)
da_sm_profile = np.where(sm_mask, da_sm_profile, np.nan)
do_sm_profile = np.where(sm_mask, do_sm_profile, np.nan)

In [ ]:
# Get data for month 1
month1_data = do_total_incr_cnt[0, :]

# Exclude NaNs
valid_idx = np.where(~np.isnan(month1_data))[0]
valid_values = month1_data[valid_idx]

# Get indices of the 10 highest values among valid entries
if len(valid_values) >= 10:
    top10_local_idx = np.argsort(valid_values)[-10:][::-1]
else:
    top10_local_idx = np.argsort(valid_values)[::-1]  # fewer than 10 valid

top10_tile_idx = valid_idx[top10_local_idx]

# Print results
print("Top 10 non-NaN values and tile numbers for month 1:")
for idx in top10_tile_idx:
    print(f"Tile {idx}: {month1_data[idx]}")

In [ ]:
def calculate_weighted_mean_series(data_array, area_array):
    """
    Calculate the weighted mean series for a given data array and area array.

    Parameters:
        data_array (numpy.ndarray): 2D array where each row represents data for a specific time step.
        area_array (numpy.ndarray): 1D array representing the area weights for each tile.

    Returns:
        list: Weighted mean series calculated for each time step.
    """
    wmean_series = []
    for i in range(data_array.shape[0]):
        data = data_array[i, :]
        weights = np.where(~np.isnan(data), area_array, 0.0)
        wmean = np.nansum(data * area_array) / np.nansum(weights)
        wmean_series.append(wmean)
    return wmean_series

# Example of usage
weighted_mean_series = calculate_weighted_mean_series(ol_snowmelt, area)
print(weighted_mean_series)

In [ ]:
# Extract the required variables for the open loop
ol_precip_ts = calculate_weighted_mean_series(ol_precip, area)
ol_runoff_ts = calculate_weighted_mean_series(ol_runoff, area)
ol_evap_ts = calculate_weighted_mean_series(ol_evap, area)
ol_base_ts = calculate_weighted_mean_series(ol_baseflow, area)
ol_tws_ts = calculate_weighted_mean_series(ol_tws, area)
ol_delta_tws_ts = ol_tws_ts - np.roll(ol_tws_ts, 1)
ol_soil_moisture_ts = calculate_weighted_mean_series(ol_soil_moisture, area)
ol_sm_profile_ts = calculate_weighted_mean_series(ol_sm_profile, area)
ol_snowmelt_ts = calculate_weighted_mean_series(ol_snowmelt, area)

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(time_array, ol_precip_ts, label='Precipitation (mm/month)', linestyle='-', color='blue')
plt.plot(time_array, ol_runoff_ts, label='Runoff (mm/month)', linestyle='--', color='green')
plt.plot(time_array, ol_evap_ts, label='Evaporation (mm/month)', linestyle='-.', color='orange')
plt.plot(time_array, ol_base_ts, label='Baseflow (mm/month)', linestyle=':', color='black')
plt.plot(time_array, ol_delta_tws_ts, label='Change in TWS (mm/month)', linestyle=':', color='red')

plt.xlabel('Time')
plt.ylabel('Values')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Open Loop Time Series')
plt.grid(True)
plt.legend()
plt.show()

# Calculate water balance as the sum of precipitation, evaporation, runoff, and baseflow
ol_water_balance_ts = np.array(ol_precip_ts) - np.array(ol_evap_ts) - np.array(ol_runoff_ts) - np.array(ol_base_ts)

# Plot the water balance and delta TWS time series
plt.figure(figsize=(12, 6))
plt.plot(time_array, ol_water_balance_ts, label='Water Balance (mm/month)', linestyle='-', color='green')
plt.plot(time_array, ol_delta_tws_ts, label='Delta TWS (mm/month)', linestyle='--', color='red')

plt.xlabel('Time')
plt.ylabel('Values')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Water Balance and Delta TWS Time Series')
plt.grid(True)
plt.legend()
plt.show()

# Calculate cumulative values for precipitation, evaporation, runoff, and baseflow
ol_cumulative_precip = np.cumsum(ol_precip_ts)
ol_cumulative_evap = np.cumsum(ol_evap_ts)
ol_cumulative_runoff = np.cumsum(ol_runoff_ts)
ol_cumulative_baseflow = np.cumsum(ol_base_ts)

# Plot cumulative values
plt.figure(figsize=(12, 6))
plt.plot(time_array, ol_cumulative_precip, label='Cumulative Precipitation (mm)', linestyle='-', color='blue')
plt.plot(time_array, ol_cumulative_evap, label='Cumulative Evaporation (mm)', linestyle='-.', color='orange')
plt.plot(time_array, ol_cumulative_runoff, label='Cumulative Runoff (mm)', linestyle='--', color='green')
plt.plot(time_array, ol_cumulative_baseflow, label='Cumulative Baseflow (mm)', linestyle=':', color='black')
plt.plot(time_array, np.cumsum(ol_delta_tws_ts), label='Cumulative Delta TWS (mm)', linestyle=':', color='red')

plt.xlabel('Time')
plt.ylabel('Cumulative Values (mm)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Open Loop Cumulative Values')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Extract the required variables for the data assimilation (DA)
da_precip_ts = calculate_weighted_mean_series(da_precip, area)
da_runoff_ts = calculate_weighted_mean_series(da_runoff, area)
da_evap_ts = calculate_weighted_mean_series(da_evap, area)
da_base_ts = calculate_weighted_mean_series(da_baseflow, area)
da_tws_ts = calculate_weighted_mean_series(da_tws, area)
da_delta_tws_ts = da_tws_ts - np.roll(da_tws_ts, 1)
da_incr_ts = calculate_weighted_mean_series(da_total_incr_sum, area)
da_incr_cnt_ts = np.nansum(da_total_incr_cnt, axis=1)
da_soil_moisture_ts = calculate_weighted_mean_series(da_soil_moisture, area)
da_sm_profile_ts = calculate_weighted_mean_series(da_sm_profile, area)
da_snowmelt_ts = calculate_weighted_mean_series(da_snowmelt, area)

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(time_array, da_precip_ts, label='Precipitation (mm/month)', linestyle='-', color='blue')
plt.plot(time_array, da_runoff_ts, label='Runoff (mm/month)', linestyle='--', color='green')
plt.plot(time_array, da_evap_ts, label='Evaporation (mm/month)', linestyle='-.', color='orange')
plt.plot(time_array, da_base_ts, label='Baseflow (mm/month)', linestyle=':', color='black')
plt.plot(time_array, da_delta_tws_ts, label='Change in TWS (mm/month)', linestyle=':', color='red')
plt.plot(time_array, da_incr_ts, label='Total Increment (mm/month)', linestyle=':', color='purple')

plt.xlabel('Time')
plt.ylabel('Values')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Data Assimilation Time Series')
plt.grid(True)
plt.legend()
plt.show()

# Calculate water balance as the sum of precipitation, evaporation, runoff, and baseflow
da_water_balance_ts = np.array(da_precip_ts) - np.array(da_evap_ts) - np.array(da_runoff_ts) - np.array(da_base_ts)
da_water_balance_with_incr_ts = da_water_balance_ts + da_incr_ts

# Plot the water balance and delta TWS time series
plt.figure(figsize=(12, 6))
plt.plot(time_array, da_water_balance_ts, label='Water Balance (mm/month)', linestyle='-', color='green')
plt.plot(time_array, da_delta_tws_ts, label='Delta TWS (mm/month)', linestyle='--', color='red')
plt.plot(time_array, da_water_balance_with_incr_ts, label='WB with Increment (mm/month)', linestyle=':', color='purple')

plt.xlabel('Time')
plt.ylabel('Values')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Water Balance and Delta TWS Time Series')
plt.grid(True)
plt.legend()
plt.show()

# Calculate cumulative values for precipitation, evaporation, runoff, and baseflow
da_cumulative_precip = np.cumsum(da_precip_ts)
da_cumulative_evap = np.cumsum(da_evap_ts)
da_cumulative_runoff = np.cumsum(da_runoff_ts)
da_cumulative_baseflow = np.cumsum(da_base_ts)

# Plot cumulative values
plt.figure(figsize=(12, 6))
plt.plot(time_array, da_cumulative_precip, label='Cumulative Precipitation (mm)', linestyle='-', color='blue')
plt.plot(time_array, da_cumulative_evap, label='Cumulative Evaporation (mm)', linestyle='-.', color='orange')
plt.plot(time_array, da_cumulative_runoff, label='Cumulative Runoff (mm)', linestyle='--', color='green')
plt.plot(time_array, da_cumulative_baseflow, label='Cumulative Baseflow (mm)', linestyle=':', color='black')
plt.plot(time_array, np.cumsum(da_delta_tws_ts), label='Cumulative Delta TWS (mm)', linestyle=':', color='red')
plt.plot(time_array, np.cumsum(da_incr_ts), label='Cumulative Total Increment (mm)', linestyle=':', color='purple')

plt.xlabel('Time')
plt.ylabel('Cumulative Values (mm)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Data Assimilation Cumulative Values')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Extract the required variables for the data assimilation (DO)
do_precip_ts = calculate_weighted_mean_series(do_precip, area)
do_runoff_ts = calculate_weighted_mean_series(do_runoff, area)
do_evap_ts = calculate_weighted_mean_series(do_evap, area)
do_base_ts = calculate_weighted_mean_series(do_baseflow, area)
do_tws_ts = calculate_weighted_mean_series(do_tws, area)
do_delta_tws_ts = do_tws_ts - np.roll(do_tws_ts, 1)
do_incr_ts = calculate_weighted_mean_series(do_total_incr_sum, area)
do_incr_cnt_ts = np.nansum(do_total_incr_cnt, axis=1)
do_soil_moisture_ts = calculate_weighted_mean_series(do_soil_moisture, area)
do_sm_profile_ts = calculate_weighted_mean_series(do_sm_profile, area)
do_snowmelt_ts = calculate_weighted_mean_series(do_snowmelt, area)

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(time_array, do_precip_ts, label='Precipitation (mm/month)', linestyle='-', color='blue')
plt.plot(time_array, do_runoff_ts, label='Runoff (mm/month)', linestyle='--', color='green')
plt.plot(time_array, do_evap_ts, label='Evaporation (mm/month)', linestyle='-.', color='orange')
plt.plot(time_array, do_base_ts, label='Baseflow (mm/month)', linestyle=':', color='black')
plt.plot(time_array, do_delta_tws_ts, label='Change in TWS (mm/month)', linestyle=':', color='red')
plt.plot(time_array, do_incr_ts, label='Total Increment (mm/month)', linestyle=':', color='purple')

plt.xlabel('Time')
plt.ylabel('Values')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Water balance components')
plt.grid(True)
plt.legend()
plt.show()

# Calculate water balance as the sum of precipitation, evaporation, runoff, and baseflow
do_water_balance_ts = np.array(do_precip_ts) - np.array(do_evap_ts) - np.array(do_runoff_ts) - np.array(do_base_ts)
do_water_balance_with_incr_ts = do_water_balance_ts + np.array(do_incr_ts)

# Plot the water balance and delta TWS time series
plt.figure(figsize=(12, 6))
plt.plot(time_array, do_water_balance_ts, label='Water Balance (mm/month)', linestyle='-', color='green')
plt.plot(time_array, do_delta_tws_ts, label='Delta TWS (mm/month)', linestyle='--', color='red')
plt.plot(time_array, do_water_balance_with_incr_ts, label='WB with Increment (mm/month)', linestyle=':', color='purple')

plt.xlabel('Time')
plt.ylabel('Values')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Water Balance and Delta TWS Time Series')
plt.grid(True)
plt.legend()
plt.show()

# Calculate cumulative values for precipitation, evaporation, runoff, and baseflow
do_cumulative_precip = np.cumsum(do_precip_ts)
do_cumulative_evap = np.cumsum(do_evap_ts)
do_cumulative_runoff = np.cumsum(do_runoff_ts)
do_cumulative_baseflow = np.cumsum(do_base_ts)

# Plot cumulative values
plt.figure(figsize=(12, 6))
plt.plot(time_array, do_cumulative_precip, label='Cumulative Precipitation (mm)', linestyle='-', color='blue')
plt.plot(time_array, do_cumulative_evap, label='Cumulative Evaporation (mm)', linestyle='-.', color='orange')
plt.plot(time_array, do_cumulative_runoff, label='Cumulative Runoff (mm)', linestyle='--', color='green')
plt.plot(time_array, do_cumulative_baseflow, label='Cumulative Baseflow (mm)', linestyle=':', color='black')
plt.plot(time_array, np.cumsum(do_delta_tws_ts), label='Cumulative Delta TWS (mm)', linestyle=':', color='red')
plt.plot(time_array, np.cumsum(do_incr_ts), label='Cumulative Total Increment (mm)', linestyle=':', color='purple')

plt.xlabel('Time')
plt.ylabel('Cumulative Values (mm)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Data Assimilation Old QC Cumulative Values')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Define a function to plot time series
def plot_time_series(time_array, ol_ts, da_ts, do_ts, title, ylabel, labels, linestyles):
    default_colors = ['blue', 'orange', 'green']  # Default colors for OL, DA 0, and DA -10
    plt.figure(figsize=(12, 6))
    plt.plot(time_array, ol_ts, label=f"{labels[0]} (mean: {np.mean(ol_ts):.6f})", linestyle=linestyles[0])
    plt.plot(time_array, da_ts, label=f"{labels[1]} (mean: {np.mean(da_ts):.6f})", linestyle=linestyles[1])
    plt.plot(time_array, do_ts, label=f"{labels[2]} (mean: {np.mean(do_ts):.6f})", linestyle=linestyles[2], marker='o', markersize=3)  
    plt.xlabel('Time')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.show()

# Precipitation
plot_time_series(
    time_array, ol_precip_ts, da_precip_ts, do_precip_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Precipitation Time Series',
    ylabel='Precipitation (mm/month)',
    labels=['OL Precipitation', 'DA 0 Precipitation', 'DA -10 Precipitation'],
    linestyles=['--', '-', ':']
)

# snowmelt
plot_time_series(
    time_array, ol_snowmelt_ts, da_snowmelt_ts, do_snowmelt_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Snowmelt Time Series',
    ylabel='Snowmelt (mm/month)',
    labels=['OL Snowmelt', 'DA minSWE Snowmelt', 'DA ght Snowmelt'],
    linestyles=['--', '-', ':']
)

# Evaporation
plot_time_series(
    time_array, ol_evap_ts, da_evap_ts, do_evap_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Evaporation Time Series',
    ylabel='Evaporation (mm/month)',
    labels=['OL Evaporation', 'DA 0 Evaporation', 'DA -10 Evaporation'],
    linestyles=['--', '-', ':']
)

# Runoff
plot_time_series(
    time_array, ol_runoff_ts, da_runoff_ts, do_runoff_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Runoff Time Series',
    ylabel='Runoff (mm/month)',
    labels=['OL Runoff', 'DA minSWE Runoff', 'DA ght Runoff'],
    linestyles=['--', '-', ':']
)

# soil moisture
plot_time_series(
    time_array, ol_soil_moisture_ts, da_soil_moisture_ts, do_soil_moisture_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Soil Moisture Time Series',
    ylabel='Soil Moisture (mm 3 / mm3)',
    labels=['OL Soil Moisture', 'DA 0 Soil Moisture', 'DA -10 Soil Moisture'],
    linestyles=['--', '-', ':']
)
# Soil moisture profile
plot_time_series(
    time_array, ol_sm_profile_ts, da_sm_profile_ts, do_sm_profile_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Soil Moisture Profile Time Series',
    ylabel='Soil Moisture Profile (mm3 / mm3)',
    labels=['OL Soil Moisture Profile', 'DA minSWE Soil Moisture Profile', 'DA ght Soil Moisture Profile'],
    linestyles=['--', '-', ':']
)


# Baseflow
plot_time_series(
    time_array, ol_base_ts, da_base_ts, do_base_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Baseflow Time Series',
    ylabel='Baseflow (mm/month)',
    labels=['OL Baseflow', 'DA 0 Baseflow', 'DA -10 Baseflow'],
    linestyles=['--', '-', ':']
)

# Increments
plot_time_series(
    time_array, np.zeros_like(do_incr_ts), da_incr_ts, do_incr_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Total Increments Time Series',
    ylabel='Increments (mm/month)',
    labels=['OL Increments', 'DA minSWE Increments', 'DA ght Increments'],
    linestyles=['--', '-', ':']
)

# Cumulative increments
plot_time_series(
    time_array, np.cumsum(np.zeros_like(do_incr_ts)), np.cumsum(da_incr_ts), np.cumsum(do_incr_ts),
    title=f'{expt_name} {start_date_str} - {end_date_str}: Cumulative Increments',
    ylabel='Cumulative Increments (mm)',
    labels=['OL Cumulative Increments', 'DA 0 Cumulative Increments', 'DA -10 Cumulative Increments'],
    linestyles=['--', '-', ':']
)

# Total Water Storage (TWS)
plot_time_series(
    time_array, ol_tws_ts, da_tws_ts, do_tws_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Total Water Storage Time Series',
    ylabel='TWS (mm)',
    labels=['OL TWS', 'DA 0 TWS', 'DA -10 TWS'],
    linestyles=['--', '-', ':']
)

# Delta TWS
plot_time_series(
    time_array, ol_delta_tws_ts, da_delta_tws_ts, do_delta_tws_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Delta TWS Time Series',
    ylabel='Delta TWS (mm/month)',
    labels=['OL Delta TWS', 'DA 0 Delta TWS', 'DA -10 Delta TWS'],
    linestyles=['--', '-', ':']
)

# Water Balance
plot_time_series(
    time_array, ol_water_balance_ts, da_water_balance_ts, do_water_balance_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Water Balance Time Series',
    ylabel='Water Balance (mm/month)',
    labels=['OL Water Balance', 'DA 0 Water Balance', 'DA -10 Water Balance'],
    linestyles=['--', '-', ':']
)

# Cumulative water balance
plot_time_series(
    time_array, np.cumsum(ol_water_balance_ts), np.cumsum(da_water_balance_ts), np.cumsum(do_water_balance_ts),
    title=f'{expt_name} {start_date_str} - {end_date_str}: Cumulative Water Balance',
    ylabel='Cumulative Water Balance (mm)',
    labels=['OL Cumulative Water Balance', 'DA 0 Cumulative Water Balance', 'DA -10 Cumulative Water Balance'],
    linestyles=['--', '-', ':']
)

# Cumulative water balance with increments
plot_time_series(
    time_array, np.cumsum(ol_water_balance_ts), np.cumsum(da_water_balance_with_incr_ts), np.cumsum(do_water_balance_with_incr_ts),
    title=f'{expt_name} {start_date_str} - {end_date_str}: Cumulative Water Balance with Increments',
    ylabel='Cumulative Water Balance (mm)',
    labels=['OL Cumulative Water Balance', 'DA 0 Cumulative Water Balance', 'DA -10 Cumulative Water Balance'],
    linestyles=['--', '-', ':']
)

# Plot increment counts
plot_time_series(
    time_array, np.zeros_like(do_incr_cnt_ts), da_incr_cnt_ts, do_incr_cnt_ts,
    title=f'{expt_name} {start_date_str} - {end_date_str}: Increment Counts',
    ylabel='Increment Counts',
    labels=['OL Increment Counts', 'DA minSWE Increment Counts', 'DA ght Increment Counts'],
    linestyles=['--', '-', ':']
)

# Plot cumulative increment counts
plot_time_series(
    time_array, np.cumsum(np.zeros_like(do_incr_cnt_ts)), np.cumsum(da_incr_cnt_ts), np.cumsum(do_incr_cnt_ts),
    title=f'{expt_name} {start_date_str} - {end_date_str}: Cumulative Increment Counts',
    ylabel='Cumulative Increment Counts',
    labels=['OL Cumulative Increment Counts', 'DA 0 Cumulative Increment Counts', 'DA -10 Cumulative Increment Counts'],
    linestyles=['--', '-', ':']
)


In [ ]:
sm_mask = np.logical_or(ol_snowmelt_sum > 0, da_snowmelt_sum > 0, do_snowmelt_sum > 0)
sm_mask = sm_mask.T

map_array[:, 0] = sm_mask

plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n Snowmelt mask', '-')

In [ ]:
ol_snowmelt = np.where(sm_mask, ol_snowmelt, np.nan)
ol_ts = calculate_weighted_mean_series(ol_snowmelt, area)
mean_OL = np.nanmean(ol_ts)
da_snowmelt = np.where(sm_mask, da_snowmelt, np.nan)
da_ts = calculate_weighted_mean_series(da_snowmelt, area)
mean_DA = np.nanmean(da_ts)
do_snowmelt = np.where(sm_mask, do_snowmelt, np.nan)
do_ts = calculate_weighted_mean_series(do_snowmelt, area)
mean_DO = np.nanmean(do_ts)

plt.figure(figsize=(10, 6))
plt.plot(time_array, ol_ts, linestyle='--', label=f'OL (Mean: {mean_OL:.3f})')
plt.plot(time_array, da_ts, label=f'DA 0 (Mean: {mean_DA:.3f})')
plt.plot(time_array, do_ts, linestyle=':', marker='o', markersize=4, label=f'DA -10 (Mean: {mean_DO:.3f})')
plt.xlabel('Time')
plt.ylabel('Snow melt (mm/month)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Global monthly mean snowmelt')
plt.grid(True)
plt.legend()
plt.show()

# Calculate cumulative snowmelt for each experiment
ol_cumulative_snowmelt = np.nancumsum(ol_ts)
da_cumulative_snowmelt = np.nancumsum(da_ts)
do_cumulative_snowmelt = np.nancumsum(do_ts)

# Plot cumulative snowmelt
plt.figure(figsize=(10, 6))
plt.plot(time_array, ol_cumulative_snowmelt, linestyle='--', label=f'OL (Cumulative: {ol_cumulative_snowmelt[-1]:.2f} mm)')
plt.plot(time_array, da_cumulative_snowmelt, label=f'DA 0 (Cumulative: {da_cumulative_snowmelt[-1]:.2f} mm)')
plt.plot(time_array, do_cumulative_snowmelt, linestyle=':', marker='o', markersize=4, label=f'DA -10 (Cumulative: {do_cumulative_snowmelt[-1]:.2f} mm)')
plt.xlabel('Time')
plt.ylabel('Cumulative Snowmelt (mm)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Global cumulative snowmelt')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
ol_snowmass = np.where(sm_mask, ol_snowmass, np.nan)
ol_ts = calculate_weighted_mean_series(ol_snowmass, area)
mean_OL = np.nanmean(ol_ts)
da_snowmass = np.where(sm_mask, da_snowmass, np.nan)
da_ts = calculate_weighted_mean_series(da_snowmass, area)
mean_DA = np.nanmean(da_ts)
do_snowmass = np.where(sm_mask, do_snowmass, np.nan)
do_ts = calculate_weighted_mean_series(do_snowmass, area)
mean_DO = np.nanmean(do_ts)

plt.figure(figsize=(10, 6))
plt.plot(time_array, ol_ts, linestyle='--', label=f'OL (Mean: {mean_OL:.3f})')
plt.plot(time_array, da_ts, label=f'DA 0 (Mean: {mean_DA:.3f})')
plt.plot(time_array, do_ts, linestyle=':', marker='o', markersize=4, label=f'DA -10 (Mean: {mean_DO:.3f})')
plt.xlabel('Time')
plt.ylabel('Snow mass (Kg m$^{-2}$)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Global monthly mean snow mass')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
ol_evap = np.where(sm_mask, ol_evap, np.nan)
ol_ts = calculate_weighted_mean_series(ol_evap, area)
mean_OL = np.nanmean(ol_ts)
da_evap = np.where(sm_mask, da_evap, np.nan)
da_ts = calculate_weighted_mean_series(da_evap, area)
mean_DA = np.nanmean(da_ts)
do_evap = np.where(sm_mask, do_evap, np.nan)
do_ts = calculate_weighted_mean_series(do_evap, area)
mean_DO = np.nanmean(do_ts)

plt.figure(figsize=(10, 6))
plt.plot(time_array, ol_ts, linestyle='--', label=f'OL (Mean: {mean_OL:.3f})')
plt.plot(time_array, da_ts, label=f'DA 0 (Mean: {mean_DA:.3f})')
plt.plot(time_array, do_ts, linestyle=':', marker='o', markersize=4, label=f'DA -10 (Mean: {mean_DO:.3f})')
plt.xlabel('Time')
plt.ylabel('Evaporation (mm/month)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Global monthly mean evaporation')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
ol_runoff = np.where(sm_mask, ol_runoff, np.nan)
ol_ts = calculate_weighted_mean_series(ol_runoff, area)
mean_OL = np.nanmean(ol_ts)
da_runoff = np.where(sm_mask, da_runoff, np.nan)
da_ts = calculate_weighted_mean_series(da_runoff, area)
mean_DA = np.nanmean(da_ts)
do_runoff = np.where(sm_mask, do_runoff, np.nan)
do_ts = calculate_weighted_mean_series(do_runoff, area)
mean_DO = np.nanmean(do_ts)

plt.figure(figsize=(10, 6))
plt.plot(time_array, ol_ts, linestyle='--', label=f'OL (Mean: {mean_OL:.3f})')
plt.plot(time_array, da_ts, label=f'DA 0 (Mean: {mean_DA:.3f})')
plt.plot(time_array, do_ts, linestyle=':', marker='o', markersize=4, label=f'DA -10 (Mean: {mean_DO:.3f})')
plt.xlabel('Time')
plt.ylabel('Runoff (mm/month)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Global monthly mean runoff')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
ol_scf = np.where(sm_mask, ol_scf, np.nan)
ol_ts = calculate_weighted_mean_series(ol_scf, area)
mean_OL = np.nanmean(ol_ts)
da_scf = np.where(sm_mask, da_scf, np.nan)
da_ts = calculate_weighted_mean_series(da_scf, area)
mean_DA = np.nanmean(da_ts)
do_scf = np.where(sm_mask, do_scf, np.nan)
do_ts = calculate_weighted_mean_series(do_scf, area)
mean_DO = np.nanmean(do_ts)

plt.figure(figsize=(10, 6))
plt.plot(time_array, ol_ts, linestyle='--', label=f'OL (Mean: {mean_OL:.3f})')
plt.plot(time_array, da_ts, label=f'DA 0 (Mean: {mean_DA:.3f})')
plt.plot(time_array, do_ts, linestyle=':', marker='o', markersize=4, label=f'DA -10 (Mean: {mean_DO:.3f})')
plt.xlabel('Time')
plt.ylabel('Fractional Area of Snow (SCF)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Global monthly mean SCF')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
do_soil_moisture = np.where(sm_mask, do_soil_moisture, np.nan)
do_ts = calculate_weighted_mean_series(do_soil_moisture, area)
mean_DO = np.nanmean(do_ts)
da_soil_moisture = np.where(sm_mask, da_soil_moisture, np.nan)
da_ts = calculate_weighted_mean_series(da_soil_moisture, area)
mean_DA = np.nanmean(da_ts)
ol_soil_moisture = np.where(sm_mask, ol_soil_moisture, np.nan)
ol_ts = calculate_weighted_mean_series(ol_soil_moisture, area)
mean_OL = np.nanmean(ol_ts)

plt.figure(figsize=(10, 6))
plt.plot(time_array, ol_ts, linestyle='--', label=f'OL (Mean: {mean_OL:.3f})')
plt.plot(time_array, da_ts, label=f'DA (Mean: {mean_DA:.3f})')
plt.plot(time_array, do_ts, linestyle=':', marker='o', markersize=4, label=f'DA old QC (Mean: {mean_DO:.3f})')
plt.xlabel('Time')
plt.ylabel('Soil Moisture (m3/m3)')
plt.title(f'{expt_name} {start_date_str} - {end_date_str}: Global monthly mean soil moisture')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:

# Map percent change in snowmelt
percent_change_snowmelt = (da_snowmelt_sum - ol_snowmelt_sum) / ol_snowmelt_sum
percent_change_snowmelt = np.where(np.isfinite(percent_change_snowmelt), percent_change_snowmelt, np.nan)  # Handle division by zero or NaN
percent_change_snowmelt = percent_change_snowmelt * 100

# Create a mask where percent_change_snowmelt is NaN
nan_mask = np.isnan(percent_change_snowmelt)

# Update the map array with the percent change values
map_array[:, 0] = percent_change_snowmelt

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Snowmelt', 'percent_change', 0, 800)
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Snowmelt', 'percent_change', -400, 400)

# Map percent change in snowmelt
percent_change_snowmelt = (do_snowmelt_sum - ol_snowmelt_sum) / ol_snowmelt_sum
percent_change_snowmelt = np.where(np.isfinite(percent_change_snowmelt), percent_change_snowmelt, np.nan)  # Handle division by zero or NaN
percent_change_snowmelt = percent_change_snowmelt * 100

# Create a mask where percent_change_snowmelt is NaN
nan_mask = np.isnan(percent_change_snowmelt)

# Update the map array with the percent change values
map_array[:, 0] = percent_change_snowmelt

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'Old QC {expt_name} {start_date_str} - {end_date_str}:\n % Change in Snowmelt', 'percent_change', -400, 400)

# Map percent change in snowmelt
percent_change_snowmelt = (da_snowmelt_sum - do_snowmelt_sum) / do_snowmelt_sum
percent_change_snowmelt = np.where(np.isfinite(percent_change_snowmelt), percent_change_snowmelt, np.nan)  # Handle division by zero or NaN
percent_change_snowmelt = percent_change_snowmelt * 100

# Create a mask where percent_change_snowmelt is NaN
nan_mask = np.isnan(percent_change_snowmelt)

# Update the map array with the percent change values
map_array[:, 0] = percent_change_snowmelt

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'New QC - Old QC {expt_name} {start_date_str} - {end_date_str}:\n % Change in Snowmelt', 'percent_change', -100, 100)

##############
# Map percent change in evaporation
percent_change_evap = (da_evap_sum - ol_evap_sum) / ol_evap_sum
percent_change_evap = np.where(np.isfinite(percent_change_evap), percent_change_evap, np.nan)  # Handle division by zero or NaN
percent_change_evap = percent_change_evap * 100
# Apply the NaN mask to percent_change_evap
percent_change_evap[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_evap

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Evaporation', 'percent_change', 0, 400)
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Evaporation', 'percent_change', -200, 200)

# Map percent change in evaporation
percent_change_evap = (do_evap_sum - ol_evap_sum) / ol_evap_sum
percent_change_evap = np.where(np.isfinite(percent_change_evap), percent_change_evap, np.nan)  # Handle division by zero or NaN
percent_change_evap = percent_change_evap * 100
# Apply the NaN mask to percent_change_evap
percent_change_evap[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_evap

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'Old QC {expt_name} {start_date_str} - {end_date_str}:\n % Change in Evaporation', 'percent_change', -200, 200)


#################
# Map percent change in runoff
percent_change_runoff = (da_runoff_sum - ol_runoff_sum) / ol_runoff_sum
percent_change_runoff = np.where(np.isfinite(percent_change_runoff), percent_change_runoff, np.nan)  # Handle division by zero or NaN
percent_change_runoff = percent_change_runoff * 100
percent_change_runoff[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_runoff

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Runoff', 'percent_change', 0, 400)
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Runoff', 'percent_change', -200, 200)

# Map percent change in runoff
percent_change_runoff = (do_runoff_sum - ol_runoff_sum) / ol_runoff_sum
percent_change_runoff = np.where(np.isfinite(percent_change_runoff), percent_change_runoff, np.nan)  # Handle division by zero or NaN
percent_change_runoff = percent_change_runoff * 100
percent_change_runoff[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_runoff

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'Old QC {expt_name} {start_date_str} - {end_date_str}:\n % Change in Runoff', 'percent_change', -200, 200)

#################
# Map percent change in soil moisture
percent_change_soil_moisture = (da_soil_moisture_mean - ol_soil_moisture_mean) / ol_soil_moisture_mean
percent_change_soil_moisture = np.where(np.isfinite(percent_change_soil_moisture), percent_change_soil_moisture, np.nan)  # Handle division by zero or NaN
percent_change_soil_moisture = percent_change_soil_moisture * 100
percent_change_soil_moisture[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_soil_moisture

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Soil Moisture', 'percent_change', 0, 200)
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Soil Moisture', 'percent_change', -100, 100)
plot_NA_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Soil Moisture', 'percent_change', -100, 100)


# Map percent change in soil moisture
percent_change_soil_moisture = (do_soil_moisture_mean - ol_soil_moisture_mean) / ol_soil_moisture_mean
percent_change_soil_moisture = np.where(np.isfinite(percent_change_soil_moisture), percent_change_soil_moisture, np.nan)  # Handle division by zero or NaN
percent_change_soil_moisture = percent_change_soil_moisture * 100
percent_change_soil_moisture[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_soil_moisture

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'Old QC {expt_name} {start_date_str} - {end_date_str}:\n % Change in Soil Moisture', 'percent_change', -100, 100)


#################
# Map percent change in baseflow
percent_change_baseflow = (da_baseflow_sum - ol_baseflow_sum) / ol_baseflow_sum
percent_change_baseflow = np.where(np.isfinite(percent_change_baseflow), percent_change_baseflow, np.nan)  # Handle division by zero or NaN
percent_change_baseflow = percent_change_baseflow * 100
percent_change_baseflow[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_baseflow

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Baseflow', 'percent_change', 0, 400)
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Baseflow', 'percent_change', -200, 200)

# Map percent change in soil moisture
percent_change_soil_moisture = (da_soil_moisture_mean - ol_soil_moisture_mean) / ol_soil_moisture_mean
percent_change_soil_moisture = np.where(np.isfinite(percent_change_soil_moisture), percent_change_soil_moisture, np.nan)  # Handle division by zero or NaN
percent_change_soil_moisture = percent_change_soil_moisture * 100
percent_change_soil_moisture[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_soil_moisture

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Surface Soil Moisture', 'percent_change', 0, 200)
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Surface Soil Moisture', 'percent_change', -100, 100)

# Map percent change in rootzone soil moisture
percent_change_rzmc = (da_rzmc_mean - ol_rzmc_mean) / ol_rzmc_mean
percent_change_rzmc = np.where(np.isfinite(percent_change_rzmc), percent_change_rzmc, np.nan)  # Handle division by zero or NaN
percent_change_rzmc = percent_change_rzmc * 100
percent_change_rzmc[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_rzmc

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Rootzone Soil Moisture', 'percent_change', 0, 200)
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Rootzone Soil Moisture', 'percent_change', -100, 100)

# Map percent change in profile soil moisture
percent_change_sm_profile = (da_sm_profile_mean - ol_sm_profile_mean) / ol_sm_profile_mean
percent_change_sm_profile = np.where(np.isfinite(percent_change_sm_profile), percent_change_sm_profile, np.nan)  # Handle division by zero or NaN
percent_change_sm_profile = percent_change_sm_profile * 100
percent_change_sm_profile[nan_mask] = np.nan

# Update the map array with the percent change values
map_array[:, 0] = percent_change_sm_profile

# Plot the percent change
maxval = np.nanmax(map_array[:, 0])
minval = np.nanmin(map_array[:, 0])
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Profile Soil Moisture', 'percent_change', 0, 200)
plot_global_tight_pcm(map_array, False, False, f'{expt_name} {start_date_str} - {end_date_str}:\n % Change in Profile Soil Moisture', 'percent_change', -100, 100)